# Real-time Implementation and Automation with Oanda 

--------------------------------------------------------------------------------------------------------------------

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with an Oanda Practice/Demo Account and NOT with a Live Trading Account._

------------------------------------------------------------------------------------

## Recap: Historical Data, real-time Data and Orders

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
configfile = "../../../oanda.cfg"

In [ ]:
api = tpqoa.tpqoa(configfile)

__Historical Data__

In [ ]:
from datetime import datetime

In [ ]:
import pandas_ta as ta

In [ ]:
df = api.get_history(instrument = "GBP_JPY", start = "2023-12-29", end = datetime.now().strftime('%Y-%m-%d'),
                granularity = "M1", price = "M", localize = False)


In [ ]:
df.o.rolling(20).max()

In [ ]:
df = df.rename({"o": "open", "h": "high", "l": "low", "c": "close"}, axis=1)

In [ ]:
df = df.assign(rsi=df.ta.rsi(14))

In [ ]:
df = df.assign(rsi=df.ta.rsi(14))
df = df.assign(rsi_ma=df.ta.sma(length=14, close=df.rsi))

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scattergl(
        x=df.index, y=df.rsi, mode="lines", marker=dict(color="red")
    )
)
fig.add_trace(
    go.Scattergl(
        x=df.index, y=df.rsi_ma, mode="lines", marker=dict(color="blue")
    )
)
fig.add_trace(
    go.Scattergl(
        x=df.index[df.positions == -1], y=df.loc[df.positions == -1, "rsi_ma"], mode="markers", marker=dict(color="green")
    )
)
fig       

In [ ]:
from datetime import datetime
import datetime as dt

mask = (datetime(2024, 1, 1, 23, 13, tzinfo=dt.timezone.utc) > df.index) & (df.index > datetime(2024, 1, 1, 23, 9, tzinfo=dt.timezone.utc))

In [ ]:
df.loc[mask]

In [ ]:
datetime(2024, 1, 1, 23, 11, tzinfo=dt.timezone.utc) > df.index

## Preview: A Trader Class live in action

In [ ]:
configfile = "../../../oanda-live.cfg"

In [ ]:
configfile = "../../../oanda.cfg"

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta
import time
import pandas_ta as ta

In [ ]:
import numpy as np

def retracement_positions(
    df,
    adx_short_period=14,
    adx_long_period=200,
    adx_short_threshold=20,
    atr_period=14,
    retrace_period=90,
    retrace_threshold=0.4,
    dm_diff_thresh=0
):
    highest = df.high.rolling(retrace_period, min_periods=1).max()
    lowest = df.low.rolling(retrace_period, min_periods=1).min()
    adx = df.ta.adx(length=adx_long_period)
    df = df.assign(
        adx_short=df.ta.adx(length=adx_short_period).iloc[:, 0],
        adx_long=adx.iloc[:, 0],
        atr=df.ta.atr(length=atr_period),
        dm_diff=adx.iloc[:, -2] - adx.iloc[:, -1],
        highest=highest,
        lowest=lowest,
        highest_index=None,
        lowest_index=None
    )
    filter = highest == df.high
    df.loc[filter, "highest_index"] = df.index[filter]
    filter = lowest == df.low
    df.loc[filter, "lowest_index"] = df.index[filter]
    # df.loc[:, "highest_index"] = df.highest_index.fillna(method="ffill")
    # df.loc[:, "lowest_index"] = df.lowest_index.fillna(method="ffill")

    # mod_highest_index = df.highest_index.fillna(method="bfill")
    # mod_lowest_index = df.lowest_index.fillna(method="bfill")
    df.loc[:, "highest_index"] = df.highest_index.ffill()
    df.loc[:, "lowest_index"] = df.lowest_index.ffill()

    mod_highest_index = df.highest_index.bfill()
    mod_lowest_index = df.lowest_index.bfill()
    df.loc[:, "was_new_high"] = (df.loc[mod_highest_index].high.values == df.highest.values) & df.highest_index.notna()
    df.loc[:, "was_new_low"] = (df.loc[mod_lowest_index].low.values == df.lowest.values) & df.lowest_index.notna()
    not_na = df.highest_index.notna() & df.lowest_index.notna()
    df.loc[:, "adx_was_high"] = (df.loc[mod_highest_index].adx_short.values > adx_short_threshold) & (df.loc[mod_lowest_index].adx_short.values > adx_short_threshold) & not_na
    df.loc[:, "sto"] = (df.close - df.lowest) / (df.highest - df.lowest)
    df.loc[:, "last_sto"] = df.sto.shift()

    crossover = (df.last_sto < retrace_threshold) & (df.sto > retrace_threshold)
    crossunder = (df.last_sto > retrace_threshold) & (df.sto < retrace_threshold)
    new_high_and_low = df.was_new_high & df.was_new_low
    df = df.assign(
        short_entry=(df.highest_index < df.lowest_index) & crossover & new_high_and_low & df.adx_was_high,
        long_entry=(df.lowest_index < df.highest_index) & crossunder & new_high_and_low & df.adx_was_high
    )
    direction_down = df.dm_diff < dm_diff_thresh
    df =  df.assign(
        short_entry1=df.short_entry & direction_down,
        short_entry2=df.long_entry & direction_down
    )
    df = df.assign(
        position=0
    )
    df.loc[df.short_entry1 | df.short_entry2, "position"] = -1
    df = df.assign(
        limit=df.lowest,
        stop=df.highest
    )
    return df


def retracement_trade_report(time, df):
    last_row = df.iloc[-1]
    print("{} | sto = {} | highest = {} | lowest = {} | fake long entry = {} | close = {}".format(time, last_row.sto, last_row.highest, last_row.lowest, last_row.long_entry, last_row.close))

In [ ]:
import pytz

class Trader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units,
                 stop_loss=50, indicators=[
                     "close", "sto", "last_sto", "adx_short", "dm_diff", "position", "highest", "lowest",
                     "was_new_high", "was_new_low", "adx_was_high"
                 ],
                 trade_reporter=retracement_trade_report, strategy=retracement_positions,
                 use_account_position=True,
                 **strategy_kwargs):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        self.strategy = strategy
        self.indicators = indicators
        self.trade_reporter = trade_reporter
        self.use_account_position = use_account_position
        self.strategy_kwargs = strategy_kwargs
        self.stop_loss = stop_loss
        self.account_position = None
        self.start_time = pd.Timestamp(datetime.utcnow(), tz=pytz.timezone("UTC"))
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
        self.set_positions()
    
    def get_most_recent(self, days = 5, columns=["open", "high", "low", "close"]):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).dropna(subset="c")
            df.rename(columns = {"c": "close", "o": "open", "h": "high", "l": "low"}, inplace = True)
            df = df.assign(**{self.instrument: df.close})[columns + [self.instrument]]
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        #print(self.ticks, end = " ")
        recent_tick = pd.to_datetime(time)
        mid = (ask + bid)/2
        df = pd.DataFrame(
            {
                self.instrument: mid,
                "open": mid,
                "high": mid,
                "low": mid,
                "close": mid
            }, 
            index = [recent_tick]
        )
        self.tick_data = pd.concat([self.tick_data, df])
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = pd.concat([
            self.raw_data, self.tick_data.resample(
                self.bar_length, 
                label="right"
            ).agg(
                {
                    self.instrument: "last",
                    "open": "first",
                    "high": "max",
                    "low": "min",
                    "close": "last",
                }
            ).ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        #df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        df = self.strategy(df, **self.strategy_kwargs) # start_time=self.start_time,
        #***********************************************************************
        indicator_string = f"{df.index[-1]} "
        for indicator in self.indicators:
            val = df.iloc[-1][indicator] if indicator in df.columns else None
            indicator_string += f"{indicator}: {val} "
        indicator_string += f"account position: {self.account_position}"       
        print(indicator_string, end='\r')
        
        self.data = df.copy()


    def set_positions(self):
        account_summary = self.get_account_summary(detailed=True)
        for position in account_summary["positions"]:
            if position["instrument"] == self.instrument:
                long_units = float(position["long"]["units"])
                short_units = float(position["short"]["units"])
                if long_units > 0:
                    self.account_position = long_units
                    if self.use_account_position:
                        self.position = 1
                elif short_units < 0:
                    self.account_position = short_units
                    if self.use_account_position:
                        self.position = -1
                else:
                    self.account_position = 0
                    if self.use_account_position:
                        self.position = 0
                break
    
    def execute_trades(self):
        self.set_positions()
        last_netrual = self.data["position"].iloc[-2] == 0 
        stop = self.data["stop"].iloc[-1]
        sl_distance = round(abs(self.data["close"].iloc[-1] - stop), 2) if stop else 0
        limit = round(self.data["limit"].iloc[-1], 2) if self.data["limit"].iloc[-1] else 0
        if self.data["position"].iloc[-1] == 1 and last_netrual and self.position == 0:
            order = self.create_order(
                self.instrument, self.units, suppress = True, ret = True,
                sl_distance=f"{sl_distance:.2f}", tp_price=f"{limit:.2f}"
            )
            print("order is ", order)
            self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1 and last_netrual and self.position == 0:
            print("limit", limit, "sl_distance", sl_distance)
            order = self.create_order(
                self.instrument, -self.units, suppress = True, ret = True,
                sl_distance=f"{sl_distance:.2f}", tp_price=f"{limit:.2f}"
            )
            print("order is ", order)
            self.report_trade(order, "GOING SHORT")
            self.position = -1
  
    
    def report_trade(self, order, going):
        print("order", order)
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        self.trade_reporter(time, self.data)
        print(100 * "-" + "\n")  
    

Simple Contrarian: Bar_lenght = 1min | Window = 1 (1 minute)

short

In [ ]:
trader = Trader(configfile, "GBP_JPY", bar_length = "1min", window = 1, units = 40000, retrace_threshold=0.4)

In [ ]:
while True:
    try:
        trader.get_most_recent()
        trader.stream_data(trader.instrument)#, stop = 200)
        if trader.position != 0: # if we have a final open position
            close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                              suppress = True, ret = True) 
            trader.report_trade(close_order, "GOING NEUTRAL")
            trader.position = 0
    except Exception as e:
        print("Encountered error")
        print(str(e))
        raise e
        time.sleep(2)

In [ ]:
trader.data.close.iloc[-50:]

In [ ]:
trader.data.iloc[-2:]

In [ ]:
184.811 + 0.07